# 전처리

- RMSE : 23724.4207
- 데이터 : train_csv, test_csv, subway_feature, bus_feature

In [ ]:
train_path = '/root/AI_STAGE/upstageailab-ml-competition-ml-2/1.Data/train.csv'
test_path  = '/root/AI_STAGE/upstageailab-ml-competition-ml-2/1.Data/test.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [ ]:
# 전처리 위한 데이터셋 합치기

train['data'] = 0
test['data'] = 1
concat = pd.concat([train, test])

In [ ]:
# 이름 바꾸기

concat = concat.rename(columns={'전용면적(㎡)':'전용면적'})

In [ ]:
# 본번, 부번의 경우 float로 되어있지만 범주형 변수의 의미를 가지므로 object(string) 형태로 바꾸기
concat_select['본번'] = concat_select['본번'].astype('str')
concat_select['부번'] = concat_select['부번'].astype('str')

### 결측치 탐지 및 처리

In [ ]:
# 열 전체를 넣고 스캔하기

for col in concat.columns:
    nunique = concat[col].nunique(dropna=False)
    missing_ratio = concat[col].isna().mean()
    missing_count = concat[col].isnull().sum()
    col_type = concat.dtypes[col]
    print(f"📌 {col:30} | 데이터타입: {col_type} | 고유값: {nunique:6} | 결측개수: {missing_count} | 결측률: {missing_ratio:.2%}")

In [ ]:
# 결측치는 아닌데 의미 없는 형식적 값 찾기

def detect_fake_nulls(df, suspect_values=['-', ' ', '', '.', '없음', 'nan']):
    result = {}
    for col in df.columns:
        if concat[col].dtype == 'object':
            val_counts = concat[col].value_counts(dropna=False)
            found = val_counts[val_counts.index.isin(suspect_values)]
            if not found.empty:
                result[col] = found
    return result

fake_nulls = detect_fake_nulls(concat)
for col, vals in fake_nulls.items():
    print(f"🔎 {col} 컬럼에서 의미 없는 값 발견:")
    print(vals)
    print()

In [ ]:
# 위 처럼 아무 의미도 갖지 않는 칼럼은 결측치와 같은 역할을 하므로, np.nan으로 채워 결측치로 인식되도록 합니다.
concat['도로명'] = concat['도로명'].replace(' ', np.nan)
concat['등기신청일자'] = concat['등기신청일자'].replace(' ', np.nan)
concat['거래유형'] = concat['거래유형'].replace('-', np.nan)
concat['중개사소재지'] = concat['중개사소재지'].replace('-', np.nan)
concat['k-시행사'] = concat['k-시행사'].replace('.', np.nan)
concat['k-시행사'] = concat['k-시행사'].replace('-', np.nan)
concat['k-홈페이지'] = concat['k-홈페이지'].replace('없음', np.nan)
concat['k-홈페이지'] = concat['k-홈페이지'].replace('.', np.nan)

In [ ]:
# print(concat.shape[0] * 0.8) = 902475.2000000001
# Null값이 90만개 이상인 칼럼은 삭제해보도록 하겠습니다.
print('* 결측치가 90만개 이하인 변수들 :', list(concat.columns[concat.isnull().sum() <= 900000]))     # 남겨질 변수들은 아래와 같습니다.
print('* 결측치가 90만개 이상인 변수들 :', list(concat.columns[concat.isnull().sum() >= 900000]))

# 결측치 90만개 이상인 값과 이하지만 필요없는 것 제외
# 필요없어 보이는 것 : k-전화번호, k-팩스번호, 사용허가여부, 관리비 업로드, k-수정일자

valid_cols = concat.columns[concat.isnull().sum() <= 900000]
exclude_cols = ['k-전화번호', 'k-팩스번호', '사용허가여부', '관리비 업로드', 'k-수정일자']

select = [col for col in valid_cols if col not in exclude_cols]
concat_select = concat[select]

concat.shape, concat_select.shape

In [ ]:
# 먼저, 연속형 변수와 범주형 변수를 위 info에 따라 분리해주겠습니다.
# 숫자형 분리 pd.api.types.is_numeric_dtype
con_columns = []
cat_columns = []

for column in concat_select.columns:
    if pd.api.types.is_numeric_dtype(concat_select[column]):
        con_columns.append(column)
    else:
        cat_columns.append(column)

print("연속형 변수:", con_columns)
print("범주형 변수:", cat_columns)

In [ ]:
# 전용면적별세대현황 시리즈끼리 상관관계가 있음 -> 유사하다고 판단
# 전용면적별세대현황 pca 진행

pca_cols = [
    'k-전용면적별세대현황(60㎡이하)',
    'k-전용면적별세대현황(60㎡~85㎡이하)',
    'k-85㎡~135㎡이하'
]
pca_data = concat_select[pca_cols].fillna(0)  # 혹시 모르니 결측 0으로 대체

# pca 진행할 feature 정규화
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_pca_data = scaler.fit_transform(pca_data)

# pca 적용
from sklearn.decomposition import PCA

pca = PCA(n_components=2)  # 2개 성분으로 축소
pca_components = pca.fit_transform(scaled_pca_data)

# 설명력 보기
print(pca.explained_variance_ratio_)  # 예: [0.83, 0.16]

# PCA 결과 저장
concat_select["세대면적_PCA1"] = pca_components[:, 0]
concat_select["세대면적_PCA2"] = pca_components[:, 1]

# 원본 feature 제거
concat_select.drop(columns=pca_cols, inplace=True)


In [ ]:
# 그다음으로 변수 간 상관관계 있는 feature 제거

drop_cols = ['k-관리비부과면적','k-연면적','k-전체동수']
concat_select.drop(columns=drop_cols, inplace=True)

In [ ]:
# 연속형 변수 동 단위 평균으로 결측치 채우기
# target은 건들지 말아보자

concat_select['구'] = concat_select['시군구'].str.split().str[1]
concat_select['동'] = concat_select['시군구'].str.split().str[2]

impute_targets = ['건축면적', '주차대수', '좌표X', '좌표Y', 'k-주거전용면적', 'k-전체세대수']

for col in impute_targets:
    # 1차: 동 단위 평균
    concat_select[col] = concat_select.groupby('동')[col].transform(lambda x: x.fillna(x.mean()))
    # 2차: 구 단위 평균 (동 평균이 안 되면 여기서)
    concat_select[col] = concat_select.groupby('구')[col].transform(lambda x: x.fillna(x.mean()))
    # 3차: 전체 평균 (구 평균도 안 되면 여기서)
    concat_select[col].fillna(concat_select[col].mean(), inplace=True)

In [ ]:
# 범주형 변수 결측치 채우기

cat_with_na = [
    '세대전기계약방법', 'k-시행사', '청소비관리형태', 'k-건설사(시공사)',
    '경비비관리형태', 'k-단지분류(아파트,주상복합등등)', '단지승인일',
    'k-복도유형', 'k-사용검사일-사용승인일', '단지신청일',
    'k-난방방식', 'k-관리방식', 'k-세대타입(분양형태)',
    '기타/의무/임대/임의=1/2/3/4', '아파트명', '도로명', '번지'
]

for col in cat_with_na:
   
    concat_select[col] = concat_select[col].fillna("Unknown")

In [ ]:
# 범주형 feature들 관계 보기

from scipy.stats import chi2_contingency

# Cramér's V 계산 함수
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix, correction=False)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    return np.sqrt(phi2 / min(k - 1, r - 1))

# 범주형 변수 리스트
cat_cols = cat_columns2  # 이미 나눈 리스트

In [ ]:
# 크래머스 브이 기준 관계있는 범주형 변수 제거

drop_cat_cols = [
    '본번',
    '부번',
    '도로명',
    '단지승인일',
    '단지신청일',
    'k-세대타입(분양형태)',
    'k-관리방식',
    'k-난방방식',
    'k-복도유형',
    '세대전기계약방법',
    '경비비관리형태',
    '청소비관리형태',
    '기타/의무/임대/임의=1/2/3/4',
]

# 제거 적용
concat_select.drop(columns=drop_cat_cols, inplace=True)

### 이상치 탐지 및 처리

In [ ]:
# 이상치 탐지 및 처리

def detect_outliers_iqr(df, columns, iqr_scale=1.5):
    outlier_summary = []

    for col in columns:
        if df[col].isnull().all():
            continue

        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - iqr_scale * IQR
        upper_bound = Q3 + iqr_scale * IQR

        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outlier_count = outliers.shape[0]
        outlier_ratio = outlier_count / df.shape[0] * 100

        outlier_summary.append({
            '변수': col,
            '이상치 개수': outlier_count,
            '이상치 비율(%)': round(outlier_ratio, 2)
        })

    return pd.DataFrame(outlier_summary).sort_values('이상치 비율(%)', ascending=False)


In [ ]:
# 이상치 비율 상위 5개 변수 추출
top_outlier_cols = outlier_df['변수'].head(5)

# pca 설명력이 높지도 않고 이상치도 많아서 pca 제거
# 원본 k-전용면적별세대현황(60㎡~85㎡이하) 가져오기

# 1. PCA로 만든 feature 제거
pca_cols = ['세대면적_PCA1', '세대면적_PCA2']
concat_select.drop(columns=pca_cols, inplace=True, errors='ignore')

# 2. 원본에서 특정 변수만 가져와서 추가
selected_feature = 'k-전용면적별세대현황(60㎡~85㎡이하)'
concat_select[selected_feature] = concat[selected_feature]

In [ ]:
# concat_select가 아닌 concat에서 가져와서 결측치 확인
# concat에서 가져온 이유 : concat_select에서 pca하면서 feature 지워버림

concat_select['k-전용면적별세대현황(60㎡~85㎡이하)'].isnull().sum()

# 결측치는 동/구/전체 평균으로 채우기

impute2_targets = ['k-전용면적별세대현황(60㎡~85㎡이하)']

for col in impute2_targets:
    # 1차: 동 단위 평균
    concat_select[col] = concat_select.groupby('동')[col].transform(lambda x: x.fillna(x.mean()))
    # 2차: 구 단위 평균 (동 평균이 안 되면 여기서)
    concat_select[col] = concat_select.groupby('구')[col].transform(lambda x: x.fillna(x.mean()))
    # 3차: 전체 평균 (구 평균도 안 되면 여기서)
    concat_select[col].fillna(concat_select[col].mean(), inplace=True)

In [ ]:
# 확인
# 리스트 초기화
con_columns_final = []
cat_columns_final = []

# concat_select 기준으로 분리
for col in concat_select.columns:
    if pd.api.types.is_numeric_dtype(concat_select[col]):
        con_columns_final.append(col)
    else:
        cat_columns_final.append(col)

In [ ]:
# 다시 IQR 확인

def detect_outliers_iqr(df, columns, iqr_scale=1.5):
    outlier_summary = []

    for col in columns:
        if df[col].isnull().all():
            continue

        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - iqr_scale * IQR
        upper_bound = Q3 + iqr_scale * IQR

        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outlier_count = outliers.shape[0]
        outlier_ratio = outlier_count / df.shape[0] * 100

        outlier_summary.append({
            '변수': col,
            '이상치 개수': outlier_count,
            '이상치 비율(%)': round(outlier_ratio, 2)
        })

    return pd.DataFrame(outlier_summary).sort_values('이상치 비율(%)', ascending=False)


In [ ]:
# 이상치 비율 상위 5개 변수 추출
top_outlier_cols = outlier_df['변수'].head(8)

# 이상치 제거 대신 클립 방식 선택

def clip_iqr(df, columns, k=1.5):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - k * IQR
        upper = Q3 + k * IQR
        df[col] = df[col].clip(lower, upper)
    return df

clip_cols = ['건축면적', '전용면적', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-전체세대수', '주차대수']

concat_select = clip_iqr(concat_select, clip_cols)